In [2]:
#convert single jpeg to tiff
from PIL import Image
from osgeo import gdal, osr
import os 
import pandas as pd

In [5]:
#batch convert jpeg to tiff
folder = "/Users/Zack/Desktop/farmsite/true positive/"
output = "/Users/Zack/Desktop/farmsite/detections_new/"
files = os.listdir(folder)

file = "/Users/Zack/Desktop/farmsite/detections_new.csv"
detections = pd.read_csv(file, delimiter=',', header=0)
print(len(detections))
detections.head()

58


,image
0,"image_(18, 35.64822, 23.70826, 40.21038, 23.77..."
1,"image_(18, 36.0586, 27.802, 39.67483, 19.85218..."
2,"image_(18, 36.43075, 27.29309, 39.11813, 23.15..."
3,"image_(18, 36.56755, 26.27227, 38.94161, 24.55..."
4,"image_(18, 36.58337, 27.86255, 39.15574, 23.07..."


In [6]:
for i in range(len(detections)):   
    name = detections['image'][i]
    im = Image.open(folder + name)
    name = name[:-5] + '.tiff'
    im.save(output + name)     

In [112]:
#georef function
def georef(src_filename, dst_filename, uperleftx, uperlefty):
    # Opens source dataset
    src_ds = gdal.Open(src_filename)
    format = "GTiff"
    driver = gdal.GetDriverByName(format)

    # Open destination dataset
    dst_ds = driver.CreateCopy(dst_filename, src_ds, 0)

    # Specify raster location through geotransform array
    # (uperleftx, scalex, skewx, uperlefty, skewy, scaley)
    # Scale = size of one pixel in units of raster projection
    # this example below assumes 100x100
    #gt = [uperleftx, .0000054, 0, uperlefty, 0, -.0000042]
    gt = [uperleftx, 0.000002986 , 0, uperlefty, 0, -.0000042]
    #0.5972

    # Set location
    dst_ds.SetGeoTransform(gt)

    # Get raster projection
    epsg = 4326
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(epsg)
    dest_wkt = srs.ExportToWkt()

    # Set projection
    dst_ds.SetProjection(dest_wkt)

    # Close files
    dst_ds = None
    src_ds = None

In [115]:
#batch geref
src_folder = "/Users/Zack/Desktop/detections_new/"
dst_folder = "/Users/Zack/Desktop/detections_new_georef/"
filename = os.listdir(src_folder)

for i in filename:
    src_filename = src_folder+i
    dst_filename = dst_folder+i
    uperleftx = float((i.split(","))[4]) #from filename
    uperlefty = float((i.split(","))[3]) #from filename
    georef(src_filename, dst_filename, uperleftx, uperlefty)